In [1]:
import sys
sys.path

['F:\\work\\python\\ml\\chap02',
 'E:\\Program Files\\Python37\\python37.zip',
 'E:\\Program Files\\Python37\\DLLs',
 'E:\\Program Files\\Python37\\lib',
 'E:\\Program Files\\Python37',
 '',
 'E:\\Program Files\\Python37\\lib\\site-packages',
 'E:\\Program Files\\Python37\\lib\\site-packages\\win32',
 'E:\\Program Files\\Python37\\lib\\site-packages\\win32\\lib',
 'E:\\Program Files\\Python37\\lib\\site-packages\\Pythonwin',
 'E:\\Program Files\\Python37\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\pky94\\.ipython']

In [2]:
import joblib
import os
import numpy as np
import pandas as pd
from housing.housingprep import CombinedAttributesAdder, housing_stratified_shuffle_split

폴더_경로 = "./housing"

데이터셋_폴더_경로 = os.path.join(폴더_경로, "datasets")
데이터셋_파일_경로 = os.path.join(데이터셋_폴더_경로, "housing.csv")
housing = pd.read_csv(데이터셋_파일_경로)

In [3]:
housing_train_X, housing_train_y, housing_test_X, housing_test_y = housing_stratified_shuffle_split(housing)

In [4]:
모델_폴더_경로 = os.path.join(폴더_경로, "models")
변환기_경로 = os.path.join(모델_폴더_경로, "transformer.pkl")
load_pipeline = joblib.load(변환기_경로)

In [5]:
housing_prepared = load_pipeline.fit_transform(housing_train_X)

In [6]:
모델_경로 = os.path.join(모델_폴더_경로, "regressor.pkl")
load_model = joblib.load(모델_경로)

In [7]:
from sklearn.metrics import mean_squared_error

housing_predictions = load_model.predict(housing_prepared)
load_rmse = mean_squared_error(housing_train_y, housing_predictions, squared=False)
load_rmse

19128.562647984956

In [8]:
from sklearn.model_selection import cross_val_score

load_rmse_scores = -cross_val_score(load_model, housing_prepared, housing_train_y,
                                    scoring="neg_root_mean_squared_error", cv=10)
load_rmse_scores.mean()

49413.27948346896

In [9]:
X_test_prepared = load_pipeline.transform(housing_test_X)
# 최종 예측
final_predictions = load_model.predict(X_test_prepared)

# 최종 예측값 평가
final_rmse = mean_squared_error(housing_test_y, final_predictions, squared=False)

In [10]:
final_rmse

47730.22690385927

In [11]:
전체_파이프라인_경로 = os.path.join(모델_폴더_경로, "full_pipeline.pkl")
full_pipeline = joblib.load(전체_파이프라인_경로)

In [12]:
full_pipeline.fit(housing_train_X, housing_train_y)

Pipeline(steps=[('preparation',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('attribs_adder',
                                                                   CombinedAttributesAdder()),
                                                                  ('std_scaler',
                                                                   StandardScaler())]),
                                                  ['longitude', 'latitude',
                                                   'housing_median_age',
                                                   'total_rooms',
                                                   'total_bedrooms',
                                                   'population', 'households',
                    

In [13]:
for e in zip(full_pipeline.predict(housing_test_X[:5]), housing_test_y[:5]):
    print(f'예측값: {e[0]: .2f}, 실제값: {e[1]: .2f}')

예측값:  495467.50, 실제값:  500001.00
예측값:  262676.70, 실제값:  240300.00
예측값:  235380.00, 실제값:  218200.00
예측값:  211883.33, 실제값:  182100.00
예측값:  135516.67, 실제값:  121300.00
